In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from collections import Counter
import re
spacy.cli.download('pl_core_news_lg')
nlp = spacy.load('pl_core_news_lg')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline
import gensim
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, explained_variance_score, max_error, mean_absolute_error, mean_squared_error, r2_score, roc_curve, roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,  LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler


✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_lg')


In [ ]:
def get_bows(data_tokens_train, data_tokens_test, data_lemmas_train, data_lemmas_test):

  #unigrams
  #tokens
  CountVec = TfidfVectorizer(ngram_range=(1,1))

  bow_uni_train = CountVec.fit_transform(data_tokens_train).toarray()
  bow_uni_test = CountVec.transform(data_tokens_test).toarray()

  #lemmas
  CountVec = TfidfVectorizer(ngram_range=(1,1))

  bowl_uni_train = CountVec.fit_transform(data_lemmas_train).toarray()
  bowl_uni_test = CountVec.transform(data_tokens_test).toarray()


  return bow_uni_train, bow_uni_test,  bowl_uni_train, bowl_uni_test

In [ ]:
def prep_features(X_train, X_test):

  scaler = StandardScaler()

  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  pca = PCA(n_components = 0.9).fit(X_train)
  X_train_pca = pca.transform(X_train)
  X_test_pca = pca.transform(X_test)

  print(pca.n_components_)

  return X_train_pca, X_test_pca


def compare_models(X_train, X_test, y_train,y_test, cm_name):

  class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
  class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
  print(class_weight_dict)

  scaler = MinMaxScaler()

  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  best_classifiers = {}

  #svc
  svc = SVC(class_weight = class_weight_dict, decision_function_shape='ovo', max_iter = 100000)
  svc = svc.fit(X_train, y_train)
  svc_pred = svc.predict(X_test)
  print('svc classification results')
  print(classification_report(y_test, svc_pred, zero_division = 1))
  best_classifiers['svc'] = svc_pred

  print('_'*200)

  #nb
  nb = GaussianNB()
  nb.class_prior_ = np.array(list(class_weight_dict.values())) / sum(class_weight_dict.values())
  nb = nb.fit(X_train, y_train)
  nb_pred = nb.predict(X_test)
  print('NB classification results')
  print(classification_report(y_test, nb_pred, zero_division = 1))
  print('_'*200)
  best_classifiers['nb'] = nb_pred

  mnb = MultinomialNB(force_alpha=True)
  mnb.fit(X_train, y_train)
  mnb_pred = mnb.predict(X_test)
  print('MNB classification results')
  print(classification_report(y_test, mnb_pred, zero_division = 1))
  print('_'*200)
  best_classifiers['mnb'] = mnb_pred

  #random forest
  rf = RandomForestClassifier(class_weight = class_weight_dict)
  rf = rf.fit(X_train, y_train)
  rf_pred = rf.predict(X_test)
  print('RF classification results')
  print(classification_report(y_test, rf_pred))
  print('_'*200)
  best_classifiers['rf'] = rf_pred



  #multilayer
  mlp = MLPClassifier(alpha = 0.05, learning_rate = 'adaptive')
  mlp.class_prior_ = np.array(list(class_weight_dict.values())) / sum(class_weight_dict.values())
  mlp = mlp.fit(X_train, y_train)
  mlp_pred = mlp.predict(X_test)
  print('mlp classification results')
  print(classification_report(y_test, mlp_pred, zero_division = 1))
  print('_'*200)
  best_classifiers['mlp'] = mlp_pred

  #dt
  dt = DecisionTreeClassifier(random_state = 0, max_depth = 7, class_weight = class_weight_dict)
  dt = dt.fit(X_train, y_train)
  dt_pred = dt.predict(X_test)
  print('dt classification results')
  print(classification_report(y_test, dt_pred, zero_division = 1))
  print('_'*200)
  best_classifiers['dt'] = dt_pred


  #sb
  ab = AdaBoostClassifier()
  ab.class_prior_ = np.array(list(class_weight_dict.values())) / sum(class_weight_dict.values())
  ab = ab.fit(X_train, y_train)
  ab_pred = ab.predict(X_test)
  print('AB classification results')
  print(classification_report(y_test, ab_pred, zero_division = 1))
  print('_'*200)
  best_classifiers['ab'] = ab_pred


  #logistic regression
  reg = LogisticRegression(max_iter = 10000, class_weight = class_weight_dict, multi_class = 'multinomial')
  reg = reg.fit(X_train, y_train)
  reg_pred = reg.predict(X_test)
  best_classifiers['reg'] = reg_pred

  print('REG classification results')
  print(classification_report(y_test, reg_pred, zero_division = 1))
  print('_'*200)

  #knn
  neigh = KNeighborsClassifier(n_neighbors = 4)
  neigh.class_prior_ = np.array(list(class_weight_dict.values())) / sum(class_weight_dict.values())
  neigh = neigh.fit(X_train_pca, y_train)
  knn_pred = neigh.predict(X_test_pca)

  best_classifiers['knn'] = knn_pred

  print('KNN classification results')
  print(classification_report(y_test, knn_pred, zero_division = 1))
  print('_'*200)

  best_classifier = input('pick best classifier: ')

  cm = confusion_matrix(y_test, best_classifiers[best_classifier])
  disp = ConfusionMatrixDisplay(confusion_matrix=cm)
  plt.figure(figsize = [10,10])
  ax = sns.heatmap(cm, annot = True, fmt = 'd', annot_kws = {'size': 20} , cmap = sns.color_palette("Blues", as_cmap=True))
  ax.xaxis.set_ticklabels(['KO', 'PiS', 'Konfederacja', 'Lewica'], fontsize = 20)
  ax.yaxis.set_ticklabels(['KO', 'PiS', 'Konfederacja', 'Lewica'], fontsize = 20)
  ax.set_xlabel('Predicted label', fontsize = 20)
  ax.set_ylabel('True label', fontsize = 20)
  plt.savefig(cm_name)
  plt.show()




# loading the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = './drive/MyDrive/Master/thesis_codes/'

In [ ]:
data_sejm = pd.read_csv(path+'data_sejm_spacy.csv')

In [ ]:
data_sejm = data_sejm.dropna(subset=['Speaker_party'])

In [ ]:
data_sejm  = data_sejm [data_sejm['Speaker_party'].isin(['KO','PiS','Konfederacja','Lewica'])]

In [ ]:
sample_data = data_sejm.sample(n = 10000)

In [ ]:
sample_data = sample_data.sample(frac=1).reset_index(drop=True)

In [ ]:
party_numbers = []
for p in sample_data['Speaker_party']:
  if p == 'KO':
    party_numbers.append(0)
  if p == 'PiS':
    party_numbers.append(1)
  if p == 'Konfederacja':
    party_numbers.append(2)
  if p == 'Lewica':
    party_numbers.append(3)

In [ ]:
sample_data['Party_tag'] = party_numbers

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sample_data['tokens'], sample_data['Party_tag'], test_size=0.2, stratify = sample_data['Party_tag']  ,random_state=42)

In [ ]:
X_trainl, X_testl, y_trainl, y_testl = train_test_split(sample_data['lemmas'], sample_data['Party_tag'], test_size=0.2, stratify = sample_data['Party_tag']  ,random_state=42)

In [ ]:
bow_uni_train, bow_uni_test, bowl_uni_train, bowl_uni_test = get_bows(X_train, X_test, X_trainl, X_testl)

In [ ]:
X_train_pca, X_test_pca = prep_features(bow_uni_train, bow_uni_test)

In [ ]:
X_trainl_pca, X_testl_pca = prep_features(bowl_uni_train, bowl_uni_test)

In [ ]:
compare_models(X_train_pca, X_test_pca, y_train,y_test, 'bow_spacy_party_sejm.jpg')

In [ ]:
compare_models(X_trainl_pca, X_testl_pca, y_trainl, y_testl, 'bowl_spacy_party_sejm.jpg')

# balanced

In [ ]:
data_pis = data_sejm.loc[data_sejm['Speaker_party'] == 'PiS'].sample(n = 2000)
data_ko = data_sejm.loc[data_sejm['Speaker_party'] == 'KO'].sample(n = 2000)
data_konf = data_sejm.loc[data_sejm['Speaker_party'] == 'Konfederacja'].sample(n = 2000)
data_lew = data_sejm.loc[data_sejm['Speaker_party'] == 'Lewica'].sample(n = 2000)
sample_data = pd.concat([data_pis, data_ko, data_konf, data_lew], ignore_index = True)

In [ ]:
sample_data = sample_data.sample(frac=1).reset_index(drop=True)

In [ ]:
party_numbers = []
for p in sample_data['Speaker_party']:
  if p == 'KO':
    party_numbers.append(0)
  if p == 'PiS':
    party_numbers.append(1)
  if p == 'Konfederacja':
    party_numbers.append(2)
  if p == 'Lewica':
    party_numbers.append(3)

In [ ]:
sample_data['Party_tag'] = party_numbers

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sample_data['tokens'], sample_data['Party_tag'], test_size=0.2, stratify = sample_data['Party_tag']  ,random_state=42)

In [ ]:
X_trainl, X_testl, y_trainl, y_testl = train_test_split(sample_data['lemmas'], sample_data['Party_tag'], test_size=0.2, stratify = sample_data['Party_tag']  ,random_state=42)

In [ ]:
bow_uni_train, bow_uni_test, bowl_uni_train, bowl_uni_test = get_bows(X_train, X_test, X_trainl, X_testl)

In [ ]:
X_train_pca, X_test_pca = prep_features(bow_uni_train, bow_uni_test)

In [ ]:
compare_models(X_train_pca, X_test_pca, y_train,y_test, 'bow_spacy_party_sejm_balanced.jpg')

In [ ]:
X_trainl_pca, X_testl_pca = prep_features(bowl_uni_train, bowl_uni_test)

In [ ]:
compare_models(X_trainl_pca, X_testl_pca, y_trainl, y_testl, 'bowl_spacy_party_sejm_balanced.jpg')